# Comparing interactive plotting packages in Python

My goal here is create the same interactive plot using a variety of different plotting packages in Python in order to test and compare the various options that other people have developed.  I'm looking for a plotting package that allows me to:

- create a plot of COVID-19 cases vs. time (while displaying the date correctly),
- zoom and pan on the plot,
- create customizable tooltips to show the data values when the mouse hovers over the plot,
- create a dropdown menu to choose the country,
- create a set of buttons to choose which columns I want to plot (for a given country), and
- export directly to a .html file for use on a personal website (without needing any other service).

After scouring the internet for the most popular plotting packages, I've decided to test this set of tools:

 - [Bokeh](https://bokeh.org/)
 - [Plotly](https://plotly.com/python/)
 - [Altair](https://altair-viz.github.io/)
 - [mpld3](http://mpld3.github.io/)
 - [matplotlib](https://matplotlib.org/) + [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)
 - [pygal](https://www.pygal.org/en/stable/) 
 - [bqplot](https://github.com/bqplot/bqplot)
 - [Streamlit](https://streamlit.io/) 
 
*Note 1 : pygal and bqplot don't have a way to create custom buttons or dropdown, unless you use ipywidgets, but I wanted to test them anyway.*

*Note 2 : you may not be able to run this entire notebook straight through because it binds a lot of data to the DOM.  If necessary, try running the section for a given visualization tool at one time (i.e., read in the data, run one visualization package, clear the kerneland repeat for another package).*

# Installation

As usual, I recommend using the [Anaconda Python](https://www.anaconda.com/products/individual) distribution and creating a new environment for this work:
```
$ conda create --name interactives python=3.9 jupyter pandas numpy matplotlib bokeh plotly altair mpld3
$ conda activate interactives
$ conda install -c conda-forge pygal lxml
$ conda install -c conda-forge ipywidgets mplcursors
$ conda install -c conda-forge voila ipympl
$ conda install -c conda-forge bqplot
$ conda install -c conda-forge streamlit
```

# Read in the data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# COVID-19 cases and deaths as a function of time for multiple countries
# df = pd.read_csv('data/WHO-COVID-19-global-data.csv') # in case the WHO server goes down
df = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')

# convert the date column to datetime objects for easier plotting and manipulation later on
df['Date_reported'] = pd.to_datetime(df['Date_reported'])

In [ ]:
# get all the available countries from the data 
availableCountries = df['Country'].unique().tolist()

In [ ]:
# I'll also want to take rolling means over 7 day intervals for each country
rollingAve = 7

# The DataFrame is already sorted by country, so I will just go through each country and append to a list
r1 = []
r2 = []
r3 = []
r4 = []
for c in availableCountries:
    usedf =  df.loc[df['Country'] == c]
    r1 += usedf['New_cases'].rolling(rollingAve).mean().to_list()
    r2 += usedf['New_deaths'].rolling(rollingAve).mean().to_list()
    r3 += usedf['Cumulative_cases'].rolling(rollingAve).mean().to_list()
    r4 += usedf['Cumulative_deaths'].rolling(rollingAve).mean().to_list()
df['New_cases_rolling'] = np.nan_to_num(r1)
df['New_deaths_rolling'] = np.nan_to_num(r2)
df['Cumulative_cases_rolling'] = np.nan_to_num(r3)
df['Cumulative_deaths_rolling'] = np.nan_to_num(r4)

df

# [Bokeh](https://bokeh.org/)

In [ ]:
from bokeh.plotting import *
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select, RadioButtonGroup, Div, HoverTool

output_notebook()
# un-comment the line below to output to an html file
output_file('bokeh_COVID.html')

In [ ]:
# start with one country (with Bokeh, I can choose any country to start with)
country = 'United States of America'
usedf = df.loc[df['Country'] == country]

# create a ColumnDataSource containing only the data I want to plot
# (Note: I only need to convert the pandas DataFrame to a ColumnDataSource because I want to manipulate it later in Javascript)
source = ColumnDataSource(
    data = dict(
        x = usedf['Date_reported'], 
        y = usedf['New_cases_rolling'], # included for the tooltips

    )
)

# create a ColumnDataSource containing all the necessary data so that I can send it to javascript for filtering
allSource = ColumnDataSource(
    data = dict(
        Date_reported = df['Date_reported'], 
        New_cases_rolling = df['New_cases_rolling'],
        New_deaths_rolling = df['New_deaths_rolling'],
        Cumulative_cases_rolling = df['Cumulative_cases_rolling'],
        Cumulative_deaths_rolling = df['Cumulative_deaths_rolling'],
        Country = df['Country']
    )
)


# define the tools you want to use
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

# define the tooltip
hover_tool = HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'),
        ( 'Count',  '@y{int}' ),
    ],
    formatters={
        '@x': 'datetime', # use 'datetime' formatter for '@x' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode = 'vline'
)

# create a new plot  
f = figure(tools = TOOLS, 
    width = 800, 
    height = 400, 
    x_range = [np.nanmin(usedf['Date_reported']), np.nanmax(usedf['Date_reported'])],
    y_range = [max(np.nanmin(usedf['New_cases_rolling']),0), np.nanmax(usedf['New_cases_rolling'])],
    x_axis_label = 'Date',
    y_axis_label = 'COVID-19 Count (' + str(rollingAve) + '-day rolling average)'
)
f.tools.append(hover_tool)

# fill the area
f.varea(x = 'x', y1 = 'y', y2 = 0,   
    source = source, 
    color = 'black', 
    alpha = 0.5
)

# draw the line
f.line('x', 'y',
    source = source, 
    color = 'black', 
    alpha = 1, 
    line_width = 2
)

# create the dropdown menu 
# (Note: Bokeh call's this select, because of the html nomenclature; there is also a difference Bokeh Dropdown)
select = Select(title = '', 
    value = country,
    options = df['Country'].unique().tolist(),
    width = 200,
    margin = (5, 5, 5, 80)
)

# create some radio buttons
radio = RadioButtonGroup(
    labels = ['Daily Cases', 'Daily Deaths', 'Cumulative Cases' ,'Cumulative Cases'], 
    active = 0,
    width = 100,
    margin = (5, 5, 5, 80)
)

# Javascript code for the callback
callback = CustomJS(
    args = dict(
        source = source, 
        allSource = allSource,
        select = select,
        radio = radio,
        ranges = dict(
            x = f.x_range, 
            y = f.y_range
        ) 
    ),
    code = 
    """
        // get the value from the dropdown
        var country = select.value;

        //convert the value from the radio button to a key
        var key = null;
        if (radio.active == 0) key = 'New_cases_rolling';
        if (radio.active == 1) key = 'New_deaths_rolling';
        if (radio.active == 2) key = 'Cumulative_cases_rolling';
        if (radio.active == 3) key = 'Cumulative_deaths_rolling';      
        
        // filter the full data set to include only those from that country
        if (key){
            var x = allSource.data.Date_reported.filter(function(d,i){return allSource.data.Country[i] == country});
            var y = allSource.data[key].filter(function(d,i){return allSource.data.Country[i] == country});

            //update the data in the plot
            source.data.x = x;
            source.data.y = y;
            source.change.emit();

            //reset the axis limits
            //note that this ... syntax may not work on all browsers
            ranges.x.start = Math.min(...x); 
            ranges.x.end =  Math.max(...x);
            ranges.y.start = Math.max(Math.min(...y),0); 
            ranges.y.end =  Math.max(...y);
        }
        
    """
)

#attach the callback 
select.js_on_change('value', callback)
radio.js_on_click(callback)


show(column([
    Div(text = '<h1>Bokeh COVID-19 Data Explorer</h1>'), 
    select,
    radio,
    f]
))

## Pros
- very versatile, lots of available options for selecting data
- smallest html file size of the tools that I tested (that can export to html)
- have access to Javascript callbacks (but can also create Python callbacks)
- some interesting tooltip options

## Cons
- need to write callback function in Javascript to enable exporting to html 
- requires more code than some other tools

# [Plotly](https://plotly.com/python/)

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Create functions to update the values

# For the time series plot
# Since there are actually 4 traces in the time plot (only 1 visible), I will need to send 4 data sets back from each function
# This one I can just call 4 times
def updateTimePlotX(co):
    use = df.loc[df['Country'] == co]
    return use['Date_reported'] 

# There may be a smarter way to do this, but I will write 4 functions here
def updateTimePlotY1(co):
    use = df.loc[df['Country'] == co]
    return use['New_cases_rolling']

def updateTimePlotY2(co):
    use = df.loc[df['Country'] == co]
    return use['New_deaths_rolling']

def updateTimePlotY3(co):
    use = df.loc[df['Country'] == co]
    return use['Cumulative_cases_rolling']

def updateTimePlotY4(co):
    use = df.loc[df['Country'] == co]
    return use['Cumulative_deaths_rolling']

In [ ]:
# I am going to create the dropdown list here and then add it to the figure below
# I will need to update the x and y data for the time series plot 
# Even though some data will not change, I will need to specify everything in this dropdown menu 

# Identify the countries to use 
# I will but The United States of America first so that it can be the default country on load (the first button)
availableCountries = df['Country'].unique().tolist()
availableCountries.insert(0, availableCountries.pop(availableCountries.index('United States of America'))) 

# create list for the country dropdown
dropdown = []
for c in availableCountries:
    dropdown.append(dict(
        args = [{'x': [updateTimePlotX(c), updateTimePlotX(c), updateTimePlotX(c), updateTimePlotX(c)],
                 'y': [updateTimePlotY1(c), updateTimePlotY2(c), updateTimePlotY3(c), updateTimePlotY4(c)]},
               ],
        label = c,
        method = 'update'
    ))

In [ ]:
# Create the trace, using Scatter to create lines and fill the region between the line and y=0.

# start with one country (I will use the first one in the list, since that is how the dropdowns are initialized)
country = availableCountries[0]
usedf = df.loc[df['Country'] == country]

# Create the figure.
fig = go.Figure()

# Add traces for each column
columns = ['New_cases_rolling', 'New_deaths_rolling', 'Cumulative_cases_rolling', 'Cumulative_deaths_rolling']
for i, c in enumerate(columns):
    visible = False
    if (i == 0):
        visible = True
        
    # Create the trace, using Scatter to create lines and fill the region between the line and y=0.
    trace = go.Scatter(x = usedf['Date_reported'], y = usedf[c], 
        mode = 'lines', # Set the mode the lines (rather than markers) to show a line.
        opacity = 1, 
        marker_color = 'black',
        fill = 'tozeroy',  # This will fill between the line and y=0.
        showlegend = False,
        name = 'COVID Count',
        hovertemplate = 'Date: %{x}<br>Number: %{y}<extra></extra>', #Note: the <extra></extra> removes the trace label.
        visible = visible
    )
    
    # Add that trace to the figure
    fig.add_trace(trace)


# Add the trace and update a few parameters for the axes.
fig.update_xaxes(title = 'Date')
fig.update_yaxes(title = 'COVID-19 Count (' + str(rollingAve) + '-day rolling average)', rangemode = 'nonnegative')#, fixedrange = True)
fig.update_layout(
    title_text = 'Plotly COVID-19 Data Explorer', # : '+ country)
    #hovermode = 'x',
    height = 550, 
    width = 850,
    title_y = 0.97,
    margin = dict(t = 140)
)

# Add the buttons and dropdown
# Note: I've seen odd behavior with adding the dropdown first and then the buttons. (e.g., the dropdown turns into many buttons)
fig.update_layout(
    updatemenus = [

        # Buttons for choosing the data to plot.
        dict(
            type = 'buttons',
            direction = 'left', # This defines what orientation to include all buttons.  'left' shows them in one row.
            buttons = list([
                dict(
                    args = [{'visible': [True, False, False, False]}], 
                    label = 'Daily Cases', 
                    method = 'restyle' 
                ),
                dict(
                    args = [{'visible': [False, True, False, False]}], 
                    label = 'Daily Deaths',
                    method = 'restyle'
                ),
                dict(
                    args = [{'visible': [False, False, True, False]}], 
                    label = 'Cumulative Cases',
                    method = 'restyle'
                ),
                dict(
                    args = [{'visible': [False, False, False, True]}], 
                    label = 'Cumulative Deaths',
                    method = 'restyle'
                )
            ]),
            showactive = True, # Highlight the active button
            # Below is for positioning
            x = 0.0, 
            xanchor = 'left',
            y = 1.15,
            yanchor = 'top'
        ),
        
        # Dropdown menu for choosing the country
        dict(
            buttons = dropdown,
                direction = 'down',
                showactive = True,
                x = 0.0,
                xanchor = 'left',
                y = 1.3,
                yanchor = 'top'
            ),
    ]
)

fig.show()

In [ ]:
fig.write_html('plotly_COVID.html')

## Pros
- Can code the callbacks  fully in Python and still export to html
- Tooltips are easy to use and look nice.
- Note that Plotly can also create nice 3D plots (while, e.g., Bokeh generally does not; but best to stick with 2D whenever possible anyway)

## Cons
- Code is cumbersome, especially when trying to link buttons and dropdowns
- Non-standard dropdown menu format is awkward (e.g., you can't type a letter in dropdown menu to skip there).
- I don’t think you can initialize the dropdown menu with a value (other than the first entry in the list).


# [Altair](https://altair-viz.github.io/)

In [ ]:
import altair as alt

In [ ]:
country = 'United States of America'
usedf = df.loc[df['Country'] == country]

# if you have a dataset larger than 5000 rows, use this line
alt.data_transformers.disable_max_rows()

# define a dropdown filter
dropdown = alt.binding_select(
    options = df['Country'].unique().tolist(),
    name = 'Country: '
)
dropdown_select = alt.selection_single(
    fields = ['Country'], 
    bind = dropdown, 
    init = {'Country': country}
)

# define radio buttons to change the plotting format
columns = ['New_cases_rolling', 'New_deaths_rolling', 'Cumulative_cases_rolling', 'Cumulative_deaths_rolling']
radio = alt.binding_radio(
    options = columns,
    labels = ['Daily Cases', 'Daily Deaths', 'Cumulative Cases' ,'Cumulative Cases'],
    name = ' ',
)
radio_select = alt.selection_single(
    fields = ['column'], 
    bind = radio, 
    init = {'column' : 'New_cases_rolling'}
)



# create the chart
chart = alt.Chart(
    df, 
    title = 'Altair COVID-19 Data Explorer'
).transform_fold( #in order to change the column of the data that is plotted, based on the radio buttons
    columns,
    as_ = ['column', 'value']
).mark_area(
    color = 'gray', # fill color
    line = True,
).encode(
    x = alt.X('Date_reported', 
        axis = alt.Axis(
            title = 'Date',
            format = "%b-%Y")
     ),
    y = alt.Y('value:Q',#'New_cases_rolling',
        axis = alt.Axis(title = 'COVID-19 Count (' + str(rollingAve) + '-day rolling average)')
    ),
    tooltip = [
        alt.Tooltip('Date_reported', title = 'Date'), 
        alt.Tooltip('value:Q', title = 'Count')
    ],
    color = alt.value('black'), # line color
).properties(
    width = 800,
    height = 400
).configure_title(
    fontSize = 24
).add_selection( 
     radio_select
).add_selection(
    dropdown_select
).transform_filter( 
    dropdown_select & radio_select
).interactive() #allow zoom and pan


# kludgy fix to format the radio buttons in a vertical line (seems exceedingly awkward to need this!)
# Also note that you'd have to include this also in the html file
# https://stackoverflow.com/questions/59025953/on-an-altair-plot-can-you-change-the-location-that-a-selection-e-g-dropdown
from IPython.display import HTML
display(HTML("""
    <style>
    .vega-bind {
        width:130px;
    }
    </style>
"""))
display(chart)

In [ ]:
# Note: the interactions are much faster in saved html file than in notebook
chart.save('altair_COVID.html')

## Pros
- Minimal coding with interesting syntax
- Fully coded in Python and can be exported to html

## Cons

- Doesn't look as nice as bokeh or plotly out of the box.  Though I could probably style it somehow (either here or in the html later on)

# [mpld3](http://mpld3.github.io/)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import mpld3
from mpld3 import plugins as mpld3_plugins

mpld3.enable_notebook()
%matplotlib inline

In [ ]:
# trying to work from here : http://mpld3.github.io/notebooks/sliderPlugin.html
# Note: online examples provide ways to link Javascript with Python callbacks, but if you do that, you can't export to html.
#   Since you have to write so much javascript here anyway, why not just send the data to javascript and do all manipulations 
#   there.  That will also allow linking many different buttons together.  

class mpld3CustomInteractive(mpld3.plugins.PluginBase):
    ''' 
        Add buttons and a dropdown.  
    '''

    JAVASCRIPT = '''
        mpld3.register_plugin('CustomInteractive', CustomInteractivePlugin);
        CustomInteractivePlugin.prototype = Object.create(mpld3.Plugin.prototype);
        CustomInteractivePlugin.prototype.constructor = CustomInteractivePlugin;
        CustomInteractivePlugin.prototype.requiredProps = ['idline', 'idpoints', 'idfilled', 'columns', 'labels',  
            'countries', 'column', 'country', 'tooltipFormat', 'input_data'];
        CustomInteractivePlugin.prototype.defaultProps = {}

        function CustomInteractivePlugin(fig, props){
            mpld3.Plugin.call(this, fig, props);
        };

        CustomInteractivePlugin.prototype.draw = function(){
                    
            // get the line object and the x data values
            var line = mpld3.get_element(this.props.idline);

            // get the points object
            var points = mpld3.get_element(this.props.idpoints);
            
            // get the filled region
            var filled = mpld3.get_element(this.props.idfilled);

            // store the axes object
            var axes = this.fig.axes[0];
            
            // store the columns that we will plot on the y axis
            var columns = this.props.columns;
            var column = this.props.column;
            
            // store the labels that we will use for each column (on the buttons)
            var labels = this.props.labels;
              
            // store the input data
            var input_data = JSON.parse(this.props.input_data);
            
            // store the countries
            var countries = this.props.countries;
            var country = this.props.country;
            
            // store the tooltip format
            var tooltipFormat = this.props.tooltipFormat;
            
            // Create buttons and dropdown
            // I could style this better, but I'll keep it simple
            var div = d3.select('#' + this.fig.figid); 
            div.append('div').selectAll('.myButton').data(columns).enter()
                .append('button')
                    .attr('class', function (d) { return 'myButton ' + d;})
                    // highlight the selected column
                    .style('background-color', function(d){ 
                        if (d == column) return 'white';
                        return 'silver';
                    })
                    .text(function(d,i){ return labels[i]})
                    .on('click', function(d) {
                        column = d;
                        // highlight the selected column
                        d3.selectAll('.myButton').style('background-color','silver');
                        d3.select('.' + d).style('background-color','white');
                        // update the plot
                        callback(); 
                    });            

            div.append('div')
                .style('margin-top', '10px')
                .append('select')
                    .on('change', function(){
                        console.log('change', this.value)
                        country = this.value;
                        callback();
                    })
                    .selectAll('.countryOption').data(countries).enter()
                        .append('option')
                            .text(function(d){ return d;})
                            .attr('class','countryOption')
                            .attr('value', function(d){ return d;})
                            .attr('selected', function (d){
                                if (d == country) return 'selected';
                                return null;
                            })

                    
            // callback function 
            function callback(){
                
                // replace the current data with the new values
                // take only the data from the correct country
                // since I'm working with dates, I need a conversion
                //    the input_data is in milliseconds; I need days
                var data = [];
                input_data.filter(function(d){ return d['Country'] == country;}).forEach(function(d, i){
                    data.push([d['Date_reported']/86400000, d[column]]);
                })

                // Update y axis              
                axes.ydom.domain(d3.extent(data, function(d) { return d[1] })); //for the data
                var yAxis = d3.select('#' + axes.fig.figid).select('.mpld3-yaxis');
                yAxis.transition().call(d3.axisLeft(axes.ydom));
                                              
                // Update line 
                line.elements().transition()
                    .attr('d', line.datafunc(data))
                    .style('stroke', 'black');

                // Update filled data
                // There may be a mpld3 built-in function (like datafunc) for filled lines, but I didn't see it
                //    so I will edit the path that comes out of line.datafunc to close the shape
                filled.elements().transition()
                    .attr('d', function(){
                        //var yorigin = data.at(-1)[1]; // This must need to be converted to some pixel units, but I'm not sure how
                        var yorigin = 1000; // As long as this is large, it will fill to y=0 (since there is a clipath to exlude values outside axes)
                        var xorigin = -1000; // As long as this is large, it will fill to x=0 (since there is a clipath to exlude values outside axes)
                        var path = line.datafunc(data) + 'V' + yorigin + 'H' + xorigin;
                        return path;
                    })
                    
                // Update the points that hold the tooltips
                // There may be a mpld3 built-in function (like datafunc) for points, but I didn't see it
                //    so I will simply use translate
                points.elements().transition()
                    .attr('transform', function(d, i){
                       return 'translate(' + points.ax.x(data[i][0]) + ',' + points.ax.y(data[i][1]) + ')'
                    })
                    
                // Update the tooltips
                // There doesn't appear to be a good hook to do this within mpld3.  So I will basically rewrite the tooltip here.
                // Get the last tooltip on the page (Note that each time the notebook is run, a new tooltip is added)
                //    It would be nice if the tooltip was also within the figure, or had a better way to acces it
                var foo =  d3.selectAll('.mpld3-tooltip')
                var last = foo.size() - 1;
                var tooltip = d3.select(foo.nodes()[last]); 
                
                // In case the tooltip wasn't already populated, I need to supply the format
                tooltip.html(tooltipFormat);
                
                // Select the dom elements for the x and y data
                var tooltipX = tooltip.select('tbody').select('tr').select('td');
                var tooltipY = tooltip.select('tbody').select('tr:nth-child(2)').select('td');
                
                // Now recreate the function to replace data in the tooltips
                points.elements().on('mouseover', function(d, i){
                    // make tooltip visible
                    tooltip.style('visibility', 'visible');
                    
                    // get the pixel values given the transform attribute of the points (awkward!, but hopefully correct!)
                    var transform = d3.select(this).attr('transform');
                    var foo = transform.split(',')
                    var pX = parseFloat(foo[0].replace('translate(',''));
                    var pY = parseFloat(foo[1].replace(')',''));
                    
                    // Convert these to data values
                    var dX = axes.xdom.invert(pX); 
                    var dY = axes.ydom.invert(pY);
                    
                    // Format and add them to the tooltip
                    tooltipX.text(dX.toLocaleDateString('en-US')) 
                    tooltipY.text(parseFloat(dY).toFixed(2))
                })

                
            }

        };
    '''

    def __init__(self, line, points, filled, columns, labels, countries, column, country, tooltipFormat, input_data ):
        self.dict_ = {'type': 'CustomInteractive', # this needs to be the same as the string in mpld3.register_plugin
                      'idline': mpld3.utils.get_id(line),
                      'idpoints': mpld3.utils.get_id(points, 'pts'),
                      'idfilled': mpld3.utils.get_id(filled),
                      'columns' : columns,
                      'labels' : labels,
                      'countries' : countries,
                      'column' : column,
                      'country' : country,
                      'tooltipFormat' : tooltipFormat,
                      'input_data' : input_data}
    

In [ ]:
# tooltips (http://mpld3.github.io/examples/html_tooltips.html)
def createTooltips(point):
    # Define some CSS to control our custom labels
    css = '''
        table
        {
          border-collapse: collapse;
        }
        th
        {
          color: #ffffff;
          background-color: #000000;
        }
        td
        {
          background-color: #cccccc;
        }
        table, th, td
        {
          font-family:Arial, Helvetica, sans-serif;
          border: 1px solid black;
          text-align: right;
          padding-right: 4px;
        }
    '''

    labels = []
    for i in range(len(usedf.index)):
        tmp = usedf.iloc[[i], :][['Date_reported','New_cases_rolling']].rename(
            columns = {'Date_reported':'Date ','New_cases_rolling':'Count '}
        )
        label = tmp.T
        label.iat[0,0] = label.iat[0,0].strftime('%m/%d/%Y') # to match the formatting that is most simple in javascript
        label.columns = ['COVID-19 Count']
        labels.append(str(label.to_html()))


    # Note: there appears to also be a LineHTMLTooltip plugin, but I can't get that to work
    tooltip = mpld3_plugins.PointHTMLTooltip(
        points, 
        labels,
        voffset = 10, 
        hoffset = 10, 
        css = css
    )
    
    return tooltip

In [ ]:
country = 'United States of America'
usedf = df.loc[df['Country'] == country].reset_index(drop = True)

# Create the initial plot with matplotlib
fig, ax = plt.subplots(figsize = (15, 7))
line, = ax.plot(usedf['Date_reported'], usedf['New_cases_rolling'], color = 'black')
filled = ax.fill_between(usedf['Date_reported'], usedf['New_cases_rolling'], color = 'gray')

# Format the x axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
ax.set_xlim(min(usedf['Date_reported']),max(usedf['Date_reported']))
ax.set_xlabel('Date', fontsize = 16, labelpad = 16)

# Format the y axis
ax.set_ylim(0,max(usedf['New_cases_rolling']))
ax.set_ylabel('COVID-19 Count (' + str(rollingAve) + '-day rolling average)', fontsize = 16, labelpad = 24)

# Set the title 
ax.set_title('mpld3 COVID-19 Data Explorer', fontsize = 24)

# Adjust the subplots
fig.subplots_adjust(bottom = 0.2)

# include points to anchor the tooltips to (but don't actually show them in the plot)
points, = ax.plot(usedf['Date_reported'], usedf['New_cases_rolling'], color = 'black', marker = 'o', markersize = 20, alpha = 0)

# Create the tooltips plugin
tooltips = createTooltips(points)

# Create the buttons plugin
columns = ['New_cases_rolling', 'New_deaths_rolling', 'Cumulative_cases_rolling', 'Cumulative_deaths_rolling']
labels = ['Daily Cases', 'Daily Deaths', 'Cumulative Cases' ,'Cumulative Deaths']

#buttons = mpld3CustomInteractive(line, points, filled, columns, labels, availableCountries, 'New_cases_rolling', country, df.to_json(orient = 'records'))
buttons = mpld3CustomInteractive(line, points, filled, columns, labels, availableCountries, 
    'New_cases_rolling', country, tooltips.labels[0], df[columns + ['Date_reported', 'Country']].to_json(orient = 'records'))

#buttons = mpld3CustomInteractive(line, points, filled, columns, labels, [country], 
#    'New_cases_rolling', country, tooltips.labels[0], usedf.to_json(orient = 'records'))

# Connect the plugins
mpld3_plugins.connect(fig, tooltips, buttons)

In [ ]:
file = open('mpld3_COVID.html', 'w')
mpld3.save_html(fig, file)
file.close()

## Pros

- It's cool to have direct access to D3 (including transitions)!

## Cons

- Zoom and pan tools appear to be only available in html (not in notebook)

- Double click to zoom is a bit odd (I am used to double click resetting the zoom)

- This requires a LOT of Javascript, and it's harder to debug (error messages in the browser console are not always helpful)

- Using the tooltips with this setup was not easy.  I was very tempted to simply code in the Javascript (but that would defeat the purpose of using mpld3).  Also note that a new tooltip is appended to the DOM each time the notebook is run, so you'd need to grab the last tooltip to edit it. 

- The html file is very large (~5-10 times larger than for the Bokeh and Plotly), though this may be due to the way that I formatted the data in the input json.

- Very minimal documentation.

# [Matplotlib](https://matplotlib.org/) + [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import mplcursors
from IPython.display import display, clear_output

# allow the plot to be interactive
%matplotlib notebook

In [ ]:
columns = ['New_cases_rolling', 'New_deaths_rolling', 'Cumulative_cases_rolling', 'Cumulative_deaths_rolling']
options = ['Daily Cases', 'Daily Deaths', 'Cumulative Cases' ,'Cumulative Deaths']

country = 'United States of America'
column = 'New_cases_rolling'

# create the initial plot
usedf = df.loc[df['Country'] == country].reset_index(drop = True)

# Create the initial plot with matplotlib
fig, ax = plt.subplots(figsize = (10, 5))
line, = ax.plot(usedf['Date_reported'], usedf[column], color = 'black', linewidth = 5)
filled = ax.fill_between(usedf['Date_reported'], usedf[column], color = 'gray')

# Format the x axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
ax.set_xlim(min(usedf['Date_reported']),max(usedf['Date_reported']))
ax.set_xlabel('Date', fontsize = 14, labelpad = 14)

# Format the y axis
ax.set_ylim(0,max(usedf[column]))
ax.set_ylabel('COVID-19 Count\n (' + str(rollingAve) + '-day rolling average)', fontsize = 14, labelpad = 24)

# Set the title 
ax.set_title('matplotlib+ipywidgets COVID-19 Data Explorer', fontsize = 16)

# adjust the spacing
fig.subplots_adjust(bottom = 0.15, left = 0.15)

# function to update the plot data
def updatePlot(country, column):
    # select the correct country
    usedf = df.loc[df['Country'] == country].reset_index(drop = True)
    
    # replace the data
    # for the line
    line.set_xdata(usedf['Date_reported'])
    line.set_ydata(usedf[column])
    
    # the filled region is a poly collection and required different syntax
    xData = mdates.date2num(usedf['Date_reported'])
    yData = usedf[column].values
    verts = [[x,y] for (x,y) in zip(xData, yData)]
    # close the region
    verts.append([xData[-1],0])
    verts.append([xData[0], 0])
    filled.set_verts([verts])
    
    # redraw the figure
    fig.canvas.draw()
    fig.canvas.flush_events()
    
    # rescale
    ax.set_ylim(0,np.nanmax(usedf[column]))


def changeCountry(change):
    # called when the dropdown menu changes
    global country
    country = change.new
    updatePlot(country, column)

def changeColumn(change):
    # called when the radio buttons changes
    global column
    index = options.index(change.new)
    if (index >= 0):
        column = columns[index]
        updatePlot(country, column)
    else:
        print('bad radio button input', change.new, index)

    
# create the dropdown widget
dropdown = widgets.Dropdown(
    value = country, 
    options = availableCountries, 
    description = ''
)
# when the value changes, execute the callback function "changeCountry".
dropdown.observe(changeCountry, names = 'value')

# create the radio buton and attach them
radiobuttons = widgets.RadioButtons(
    value = 'Daily Cases', 
    options = options, 
    description = ''
)
radiobuttons.observe(changeColumn, names = 'value')

# add tooltips using mplcursors
# https://mplcursors.readthedocs.io/en/stable/examples/hover.html
mplcursors.cursor(hover=True)

# a VBox container to pack widgets vertically
widgets.VBox(
    [
        dropdown, 
        radiobuttons,
    ]
)

[Voila](https://voila.readthedocs.io/en/stable/) can be used to host an interactive plot made with matplotlib and ipywidgets online.  You can't simply generate a .html file, but you could deploy the plot online following the instructions on their website.  I created a simple test notebook for that purpose : ipywidgets_COVID.ipynb

## Pros

- Most people already know matplotlib (though not necessarily ipywidgets)
- Minimal coding is needed

## Cons

- No easy way to export this to html, though Voila is helpful
- I had to use another package for tooltips (mplcursors; though it was very easy to use). 
- Doesn't look quite as good as others, though perhaps could be styled a bit better with some more time.

# [pygal](https://www.pygal.org/en/stable/)

In [ ]:
import pygal
from pygal.style import Style

from datetime import date, datetime, timedelta
from IPython.display import SVG, display

In [ ]:
country = 'United States of America'
usedf = df.loc[df['Country'] == country].reset_index(drop = True)

# Add some custom styling : https://www.pygal.org/en/latest/documentation/custom_styles.html
custom_style = Style(
    font_family = 'Helvetica, sans-serif',
    #transition = '400ms ease-in',
    #stroke_opacity = (1, 0),
    colors = ('purple', 'black',)
)

chart = pygal.DateLine(
    #show_legend = False,
    #show_dots = False,  # you need the points in order to see the tooltips
    style = custom_style,
    width = 800, height = 400, explicit_size = True,
    xrange = list(np.array([np.nanmin(usedf['Date_reported']), np.nanmax(usedf['Date_reported'])]).astype('int64')/1e9),
    # attempting to compensate for the offset in xrange
    # xrange = list(np.array([np.nanmin(usedf['Date_reported']), np.nanmax(usedf['Date_reported']) - np.timedelta64(35,'D')]).astype('int64')/1e9),
    yrange = [max(np.nanmin(usedf['New_cases_rolling']),0), np.nanmax(usedf['New_cases_rolling'])],    
    title = 'pygal COVID-19 Data Explorer',
    x_title = 'Date',
    y_title = 'COVID-19 Count (' + str(rollingAve) + '-day rolling average)',
    tooltip_border_radius = 10
)

# Get the data in the correct format (a list of tuples)
# Also convert the dates to unix-style seconds
xData = np.array(usedf['Date_reported'].values.astype('int64')/1e9, dtype='int32')
yData1 = usedf['New_cases_rolling']
yData2 = usedf['Cumulative_cases_rolling']
yData3 = usedf['New_deaths_rolling']
yData4 = usedf['Cumulative_deaths_rolling']
data1 = [(x,y) for (x,y) in zip(xData, yData1)]
data2 = [(x,y) for (x,y) in zip(xData, yData2)]
data3 = [(x,y) for (x,y) in zip(xData, yData3)]
data4 = [(x,y) for (x,y) in zip(xData, yData4)]

# Add the data
# These two have roughly the same y scale, so I can include them in the same figure, though it doesn't really make sense to 
#   only include these.  If I include the others, the yscale is too different to see the values.  But this will at least
#   give a sense of the abilities of the interactive legend.  (Click on the legend to turn on/off the different curves.)
#chart.add('Cumulative Case', data2, fill = True, show_dots = True, stroke_style = {'width':1e-5}) # looks like you can't set width to 0
#chart.add('Daily Deaths', data3, fill = True, show_dots = True, stroke_style = {'width':1e-5}) # looks like you can't set width to 0
chart.add('Cumulative Deaths', data4, fill = True, show_dots = True, stroke_style = {'width':1e-5}) # looks like you can't set width to 0
chart.add('Daily Cases', data1, fill = True, show_dots = True, stroke_style = {'width':1e-5}) # looks like you can't set width to 0

# x labels at the start of months
fac = 3
date2020 = [date(2020, fac*(m + 1), 1) for m in range(int(12/fac))]
date2021 = [date(2021, fac*(m + 1), 1) for m in range(int(12/fac))]
#date2022 = [date(2022, fac*(m + 1), 1) for m in range(1)]
chart.x_labels = date2020 + date2021


display(SVG(chart.render(disable_xml_declaration = True)))

#Note: I didn't spend more time customizing since it can't handle the level of interaction that I want

In [ ]:
# render in browser, and then you can move (or copy) and rename the file to the location and name I want
chart.render_in_browser()

## Pros

- generates SVG graphics
- fairly simple to create a plot with tooltips and an interactive legend

## Cons

- Converting the dates took some time to figure out.
- xrange is not exact.  There is still some padding that I can't figure out how to remove.  And even if I try to remove it manually, the plot just moves off of the canvas.  (There is no clipPath)
- There is no ability to set the yrange (at least that I know of).
- Interactions (tooltips and interactive legend) only work up in the HTML version (not in Jupyter).
- No way to include buttons with more sophisticated interactions (without using something like ipywidgets).
- No native way to zoom and pan (at least not that I found)

# [bqplot](https://github.com/bqplot/bqplot)

In [ ]:
from bqplot import pyplot as plt
from bqplot.interacts import PanZoom
from bqplot import DateScale, LinearScale, Tooltip

In [ ]:
columns = ['New_cases_rolling', 'New_deaths_rolling', 'Cumulative_cases_rolling', 'Cumulative_deaths_rolling']
options = ['Daily Cases', 'Daily Deaths', 'Cumulative Cases' ,'Cumulative Deaths']

country = 'United States of America'
column = 'New_cases_rolling'
label = 'Daily cases : '

usedf = df.loc[df['Country'] == country].reset_index(drop = True)



# Create the initial plot with matplotlib
fig = plt.figure(figsize = (13, 5),
    title = 'bqplot COVID-19 Data Explorer'
)
line = plt.plot(usedf['Date_reported'].to_numpy(), usedf[column].to_numpy(),
    axes_options = {'y': {'label_offset': '-50'}},
#    tooltip = tt #it looks like tooltips don't work on lines
)
line.colors = ['black']
line.stroke_width = 5
line.fill = 'bottom'
line.fill_colors = ['gray']
plt.xlabel('Date')
plt.ylabel('COVID-19 Count (' + str(rollingAve) + '-day rolling average)')

# interactions : https://github.com/bqplot/bqplot/blob/master/examples/Interactions/Interaction%20Layer.ipynb
# add the panzoom
# this doesn't appear to be working??? and also disabled the tooltips
# xs_pz = DateScale(min = min(usedf['Date_reported'].to_numpy()))
# ys_pz = LinearScale()
# panzoom = PanZoom(scales = {'x': [xs_pz], 'y': [ys_pz]})
# fig.interaction = panzoom


# for tooltips I will create a scatter plot and just don't show the points
tt = Tooltip(fields = ['name', 'y'], labels = ['Date : ', label], formats = ['','.2f'])
usedf['Datestr'] = usedf.apply(lambda row: f"{row['Date_reported']:%b %d, %Y}", axis=1)

scatter = plt.scatter(usedf['Date_reported'].to_numpy(), usedf[column].to_numpy(), 
    colors = ['#ffffff00'],
    names = usedf['Datestr'], display_names = False,
    tooltip = tt)

fig

## Pros
- A plotting tool that allows for various selections on data, tooltips and animations.

## Cons
- Documentation is lacking.
- If you want to include buttons or dropdowns, you need to use ipywidgets (or another similar package).
- I don't think there's an easy way to export to html.
- Pan and zoom don't seem to work (at least on my end).
- Pan+zoom and tooltips seem to be mutually exclusive.

# [Streamlit](https://streamlit.io/)  

Really slick.  Very easy to use.  It is essentially a wrapper for other plotting utilities (like plotly, bokeh and altair) but because of that it requires minimal code to produce something usable (and with more code it can be more customizable).

This can't be run within a Jupyter notebook.  See streamlitTest.py.  


## Pros
- Very nice looking interface.
- Minimal coding needed to create a very useful app.
- Full coding in Python

## Cons
- Can't get the html file directly (but can [deploy to github](https://docs.streamlit.io/streamlit-cloud/get-started/deploy-an-app)).
- Can't run in a Jupyter notebook.
- Minimal options available for customizing plots with streamlits wrapper functions (though you do have access to the original functions, e.g., from altiar, plotly, bokeh, to enable more customization)

# Others

- [HoloViz](https://holoviz.org/index.html) and [Panel](https://panel.holoviz.org/): used to create dashboards with Bokeh and Plotly
- [Dash](https://plotly.com/dash/) : used to create dashboards with Plotly.
- [Datapane](https://datapane.com/) : used to stitch together figures and tables created by other libraries